# Project Milestone3_DSC540

In [1]:
#Name: Bhargava Gaggainpali
#Course: DSC 540 Data Preparation
#Title: Monroe County Crash Data

#Project Milestone 3 - Web Scrapping (Holidays website)

In [2]:
# import required packages

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
from urllib.request import urlopen
from dateutil.parser import parse

In [3]:
# Define variables to support processing website data

Website_url = "https://www.opm.gov/policy-data-oversight/pay-leave/federal-holidays/#url=2015"
Website_html = urlopen(Website_url)   # open website using urlopen
soup = BeautifulSoup(Website_html, 'html.parser') # parse the data in the website
Website_all_tables = soup.find_all('table') # Finding the tables in the website
print("Total number of tables are {} ".format(len(Website_all_tables)))



Total number of tables are 17 


In [4]:
# Print the data in the website table header
Website_table_headers = []
for tx in soup.find_all('table'):
    Website_table_headers.append(tx) 

print(Website_table_headers)

[<table cellpadding="0" cellspacing="0"><tr><td class="ddNavCorner ddNavCornerTL"></td><td class="ddNavEdge ddNavEdgeTop"></td><td class="ddNavCorner ddNavCornerTR"></td></tr><tr><td class="ddNavEdge ddNavEdgeLeft"></td><td class="ddNavBody HasFeaturedContent "><div class="SecondaryNav_Wrapper" parentnavitem="/about-us/"><ul class="SecondaryNav"><li class=""><a href="/about-us/"><span>Our Agency</span></a></li>
<li class=""><a href="/about-us/our-director/"><span>Our Director</span></a></li>
<li class=""><a href="/about-us/our-inspector-general/"><span>Our Inspector General</span></a></li>
<li class=""><a href="/about-us/our-mission-role-history/"><span>Our Mission, Role &amp; History</span></a></li>
<li class=""><a href="/about-us/our-people-organization/"><span>Our People &amp; Organization</span></a></li>
<li class=""><a href="/about-us/careers-at-opm/"><span>Careers at OPM</span></a></li>
<li class=""><a href="/about-us/doing-business-with-opm/"><span>Doing Business with OPM</span>

In [5]:
# define a function to get the rows of data in the table
def tableDataText(table):      
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows

In [6]:
df = None  # define a null DataFrame

# run a loop in all the tables in website to find 2015 data and save it in dataframe
for tb in Website_all_tables:
    caption = tb.find("caption")
    if not caption is None and "2015" in caption.contents[0]:
        items = tableDataText(tb)
        df = pd.DataFrame(items[1:], columns=['Date', 'Holiday'])
        break



In [7]:
# run a loop to format the data
if not df is None:
    for i in range(df.shape[0]):
        row = df.iloc[i,]
        df.Date.iloc[i] = row['Date'].replace("*", "")
        df.Date.iloc[i] = df.Date.iloc[i] + " 2015"
        df.Date.iloc[i] = parse(df.Date.iloc[i])



In [8]:
# Print 2015 data in the dataframe.
print(df)

                  Date                              Holiday
0  2015-01-01 00:00:00                       New Year’s Day
1  2015-01-19 00:00:00  Birthday of Martin Luther King, Jr.
2  2015-02-16 00:00:00                Washington’s Birthday
3  2015-05-25 00:00:00                         Memorial Day
4  2015-07-03 00:00:00                     Independence Day
5  2015-09-07 00:00:00                            Labor Day
6  2015-10-12 00:00:00                         Columbus Day
7  2015-11-11 00:00:00                         Veterans Day
8  2015-11-26 00:00:00                     Thanksgiving Day
9  2015-12-25 00:00:00                        Christmas Day


In [9]:
df.dtypes

Date       object
Holiday    object
dtype: object

In [10]:
import sqlite3   # import sqlite3 package
conn = sqlite3.connect('DSC540_monroe_county.db')  # Connect to database
c = conn.cursor()    # create an instance



In [11]:
# Create table - HOLIDAYS
c.execute('''CREATE TABLE HOLIDAYS(
[Date] TEXT,
[Holiday] TEXT
)''')   # Create table syntax
conn.commit()  # Commit the changes

In [12]:
import pandas as pd   # Import the package pandas
# load data from Dataframe to Table.
df.to_sql('HOLIDAYS', conn, if_exists='append', index=False)